In [1]:
import cv2
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import HTML
import numpy as np

In [2]:
def RGB2GRAY_func(frame1):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_RGB2GRAY)
    return gray1
 

In [3]:
def GaussianBlur_func(gray,kernel_size):
    blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    return blur_gray

In [4]:
def Canny_func(blur_gray, low_threshold, high_threshold):
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    return edges

In [5]:
def FillPoly_func(edges, mask, vertices, ignore_mask_color):
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)
    return masked_edges


In [6]:
def HoughLinesP_func(masked_edges, rho, theta, threshold,min_line_length, max_line_gap):
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    return lines

In [7]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane.
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.

    This function draws `lines` with `color` and `thickness`.
    Lines are drawn on the image inplace (mutates the image).

    """
    # In case of error, don't draw the line(s)
    if lines is None:
        return
    if len(lines) == 0:
        return
    draw_right = True
    draw_left = True

    # Find slopes of all lines
    # But only care about lines where abs(slope) > slope_threshold
    slope_threshold = 0.6
    slopes = []
    new_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]  # line = [[x1, y1, x2, y2]]

        # Calculate slope
        if x2 - x1 == 0.:  # corner case, avoiding division by 0
            slope = 999.  # practically infinite slope
        else:
            slope = (y2 - y1) / (x2 - x1)

        # Filter lines based on slope
        if abs(slope) > slope_threshold:
            slopes.append(slope)
            new_lines.append(line)

    lines = new_lines

    # Split lines into right_lines and left_lines, representing the right and left lane lines
    # Right/left lane lines must have positive/negative slope, and be on the right/left half of the image
    right_lines = []
    left_lines = []
    for i, line in enumerate(lines):
        x1, y1, x2, y2 = line[0]
        img_x_center = img.shape[1] / 2  # x coordinate of center of image
        if slopes[i] > 0 and x1 > img_x_center and x2 > img_x_center:
            right_lines.append(line)
        elif slopes[i] < 0 and x1 < img_x_center and x2 < img_x_center:
            left_lines.append(line)

    # Run linear regression to find best fit line for right and left lane lines
    # Right lane lines
    right_lines_x = []
    right_lines_y = []

    for line in right_lines:
        x1, y1, x2, y2 = line[0]

        right_lines_x.append(x1)
        right_lines_x.append(x2)

        right_lines_y.append(y1)
        right_lines_y.append(y2)

    if len(right_lines_x) > 0:
        right_m, right_b = np.polyfit(right_lines_x, right_lines_y, 1)  # y = m*x + b
    else:
        right_m, right_b = 1, 1
        draw_right = False

    # Left lane lines
    left_lines_x = []
    left_lines_y = []

    for line in left_lines:
        x1, y1, x2, y2 = line[0]

        left_lines_x.append(x1)
        left_lines_x.append(x2)

        left_lines_y.append(y1)
        left_lines_y.append(y2)

    if len(left_lines_x) > 0:
        left_m, left_b = np.polyfit(left_lines_x, left_lines_y,1)  # y = m*x + b
    else:
        left_m, left_b = 1, 1
        draw_left = False

    # Find 2 end points for right and left lines, used for drawing the line
    # y = m*x + b --> x = (y - b)/m
    y1 = img.shape[0]
    y2 = img.shape[0] * (1 - 0.38)

    right_x1 = (y1 - right_b) / right_m
    right_x2 = (y2 - right_b) / right_m

    left_x1 = (y1 - left_b) / left_m
    left_x2 = (y2 - left_b) / left_m

    # Convert calculated end points from float to int
    y1 = int(y1)
    y2 = int(y2)
    right_x1 = int(right_x1)
    right_x2 = int(right_x2)
    left_x1 = int(left_x1)
    left_x2 = int(left_x2)

    # Draw the right and left lines on image

    if draw_left:
        cv2.line(img, (left_x1, y1), (left_x2, y2), color, thickness)
    if draw_right:
        cv2.line(img, (right_x1+1, y1), (right_x2+1, y2), color, thickness)

In [8]:
def process_image(frame):
    gray = RGB2GRAY_func(frame)
    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 3
    blur_gray = GaussianBlur_func(gray,kernel_size)
    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = Canny_func(blur_gray, low_threshold, high_threshold)

    mask = np.zeros_like(edges)
    ignore_mask_color = 255

    # This time we are defining a four sided polygon to mask
    imshape = frame.shape
    #vertices = np.array([[(0, imshape[0]), (390, 320), (540, 320), (imshape[1], imshape[0])]], dtype=np.int32)
    vertices = np.array([[(0, imshape[0]), (378, 344), (540, 320), (imshape[1], imshape[0])]], dtype=np.int32)

    masked_edges = FillPoly_func(edges, mask, vertices, ignore_mask_color)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 3 # distance resolution in pixels of the Hough grid
    theta = np.pi / 30  # angular resolution in radians of the Hough grid
    threshold = 1  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 5 # minimum number of pixels making up a line
    max_line_gap = 80  # maximum gap in pixels between connectable line segments
    line_image = np.copy(frame) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments

    lines = HoughLinesP_func(masked_edges, rho, theta, threshold,min_line_length, max_line_gap)

    # Iterate over the output "lines" and draw lines on a blank image
    draw_lines(line_image, lines)
    
    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges))

    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(frame, 0.7, line_image, 1, 0)
    #cv2.imshow('frame',frame)
    #cv2.imshow('lines_edges',lines_edges)
                        
    return lines_edges


In [9]:
def main():
    white_output = "solidYellowLeft_OutPut.mp4"
    clip1 = VideoFileClip("solidYellowLeft.mp4")
    white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(white_output, audio=False)

In [10]:
if __name__ == '__main__':
    main() 

[MoviePy] >>>> Building video solidYellowLeft_OutPut.mp4
[MoviePy] Writing video solidYellowLeft_OutPut.mp4


100%|███████████████████████████████████████▉| 681/682 [00:13<00:00, 52.28it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: solidYellowLeft_OutPut.mp4 

Wall time: 13.4 s
